<a href="https://colab.research.google.com/github/Seferovic8/chess-project/blob/main/models/Chess_notebook_from_position.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
!pip install chess

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for chess: filename=chess-1.11.1-py3-none-any.whl size=148497 sha256=c2dcad629682d7f7936c7d2e52edeae29487728f75f1e453af3c7b0854247c46
  Stored in directory: /root/.cache/pip/wheels/2e/2d/23/1bfc95db984ed3ecbf6764167dc7526d0ab521cf9a9852544e
Successfully built chess


In [ ]:
import numpy as np
import pandas as pd
import torch
from numpy import loadtxt
from io import StringIO
import chess.pgn
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from datetime import datetime


In [ ]:
#!gdown 1ruGgJKeKv2DGWgVcqmdjK9pWk0mPYSu4
!gdown 1OjVvEeEZNTvU6M2s2We6u8pFYil3dS1x

Downloading...
From (original): https://drive.google.com/uc?id=1OjVvEeEZNTvU6M2s2We6u8pFYil3dS1x
From (redirected): https://drive.google.com/uc?id=1OjVvEeEZNTvU6M2s2We6u8pFYil3dS1x&confirm=t&uuid=7bf326b4-b936-451a-ad59-6e72c30a7a83
To: /content/chess_dataset.csv
100% 2.73G/2.73G [00:12<00:00, 222MB/s]


In [ ]:
def string_to_array(string):
    string = string.replace("]", "").replace("[", "").replace("\n","").replace(" ","").replace("."," ")
    #k = '['+k+']'
    c = StringIO(string)

    ar=loadtxt(c)
    return ar.reshape((8,8,12))

In [ ]:
df = pd.read_csv("chess_dataset.csv")
df = df[:500_000]

In [ ]:
len(df)

500000

In [ ]:
df["board"]=df["board"].apply(string_to_array)

In [ ]:
df

In [ ]:
df_train, df_test = train_test_split(df, test_size=0.2)

In [ ]:
def one_hot_encode(num, length=64):
    enc = np.zeros(length)
    enc[num]=1
    return enc

In [ ]:
class ChessDataset(Dataset):
  def __init__(self, dataset):
    super(ChessDataset,self).__init__()
    self.dataset = dataset

  def __len__(self):
      return len(self.dataset)

  def __getitem__(self, idx):
    # get audio data
    row = df.loc[idx]
    board=row["board"]
    player = row['player']
    # Convert to torch tensors with appropriate data types
    board_tensor = torch.FloatTensor(board)
    player_tensor = torch.tensor(player, dtype=torch.float32)
    # Get label
    label = torch.tensor(one_hot_encode(row['fromPosition']), dtype=torch.float32)

    # Return tuple of inputs and label

    return (board_tensor, player_tensor), label

In [ ]:
train_dataset = ChessDataset(df_train)
test_dataset = ChessDataset(df_test)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

In [ ]:
for i,_ in train_loader:
  print(i[1].shape)
  break

In [ ]:
for i,d in train_dataset:
  print(i[1].view(-1,1).shape)
  break

In [ ]:
number_of_classes=64

# Create model

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
bada=[]

In [ ]:
class ChessDecoder:
      def __init__(self):
          self.pieces={
          chess.Piece(chess.PAWN, chess.WHITE): 0,
          chess.Piece(chess.PAWN, chess.BLACK): 1,
          chess.Piece(chess.ROOK, chess.WHITE): 2,
          chess.Piece(chess.ROOK, chess.BLACK): 3,
          chess.Piece(chess.KNIGHT, chess.WHITE): 4,
          chess.Piece(chess.KNIGHT, chess.BLACK): 5,
          chess.Piece(chess.BISHOP, chess.WHITE): 6,
          chess.Piece(chess.BISHOP, chess.BLACK): 7,
          chess.Piece(chess.QUEEN, chess.WHITE): 8,
          chess.Piece(chess.QUEEN, chess.BLACK): 9,
          chess.Piece(chess.KING, chess.WHITE): 10,
          chess.Piece(chess.KING, chess.BLACK): 11,
        }
          self.reverse_pieces = {value: key for key, value in self.pieces.items()}
      def getPiece(self,encoding):
          if torch.equal(encoding, torch.zeros(12).to(device)):
            return None

          return self.reverse_pieces[torch.argmax(encoding).item()]

      def getPieceEncoding(self,piece):
          if piece is not None:
              return one_hot_encode(self.pieces[piece],length=12)
          return torch.zeros(12)

      def decodeBoardPosition(self, encodedBoard,turn):
          encodedBoard=encodedBoard.reshape(64,-1)
          board=chess.Board()
          board.turn=turn
          for square in range(64):
              board.set_piece_at(square,self.getPiece(encodedBoard[square]))
          return board
      def getAllowedMoves(self,b,square,winner,en_passant=False):
              global bada
              board = self.decodeBoardPosition(b,winner)
              isLeftEdge=square%8!=0
              isRightEdge=square%8!=7
              if en_passant:
                if winner ==chess.BLACK:
                    if isLeftEdge:
                      board.ep_square = square-9
                    if isRightEdge:
                      board.ep_square = square-7
                else:
                    if isRightEdge:
                      board.ep_square = square+9
                    if isLeftEdge:
                      board.ep_square = square+7
              allowedMoves = [move.to_square for move in board.pseudo_legal_moves if move.from_square == square]
              if not allowedMoves:
                return self.getAllowedMoves(b,square,winner,en_passant=True)
              return allowedMoves

      def getMask(self,encodedBoard,from_square,player,mask_size):
          mask = torch.zeros(mask_size, 64).to(device)  # Initialize mask with zeros
          #mask.fill_(-10)

          for i,board in enumerate(encodedBoard):
              winner = bool(player[i].item())
              square=torch.argmax(from_square[i]).item()

              allowedMoves=[]
              allowedMoves = self.getAllowedMoves(board,square,winner)
             # mask[i, allowedMoves] = 0
              mask[i, allowedMoves] = 1
          return mask
      def encodeBoard(self,board):
        encodedBoard=np.zeros((64,12))
        for square in range(64):
            piece = board.piece_at(square)
            encodedBoard[square]=self.getPieceEncoding(piece)
        return encodedBoard.reshape(8,8,12)
      def getPiecesOfColor(self,board,winner):
              pieces=[]
              board = self.decodeBoardPosition(board,winner)
              for square in range(64):
                  piece = board.piece_at(square)
                  isLeftEdge=square%8!=0
                  isRightEdge=square%8!=7
                  if piece==chess.PAWN:
                    if winner ==chess.BLACK:
                        if isLeftEdge:
                          board.ep_square = square-9
                        if isRightEdge:
                          board.ep_square = square-7
                    else:
                        if isRightEdge:
                          board.ep_square = square+9
                        if isLeftEdge:
                          board.ep_square = square+7
                  if piece and piece.color == winner:
                      for move in board.legal_moves:
                          if move.from_square==square:
                              pieces.append(square)
              return pieces
      def getFromPositionMask(self,encodedBoard,player,mask_size):
           mask = torch.zeros(mask_size, 64).to(device)  # Initialize mask with zeros
          #mask.fill_(-10)

           for i,board in enumerate(encodedBoard):
              winner = bool(player[i].item())

              allowedMoves = self.getPiecesOfColor(board,winner)
              # mask[i, allowedMoves] = 0
              mask[i,allowedMoves] = 1
           return mask
      def getWinner(game):
        if game.headers['Result'] =="1-0":
            return chess.WHITE
        return chess.BLACK

      def encodeGame(self,game):
        board = chess.Board()
        data=[]
        winner = self.getWinner(game)
        # Go through each move in the game until
        # we reach the required move number
        for move in game.mainline_moves():
            if board.turn==winner:
                encodedBoard=self.encodeBoard(board)
                data.append([encodedBoard,move.from_square,move.to_square,int(winner)])
            board.push(move)
        return data

In [ ]:
class CNN(nn.Module):
  def __init__(self, n_inputs, n_outputs):
    super(CNN,self).__init__()
    self.D = n_inputs
    self.K = n_outputs
    self.conv1 = nn.Sequential(
      nn.Conv2d(self.D, 32, kernel_size=5, padding=2,stride=1),
      #nn.ReLU(),
      #nn.Conv2d(32, 64, kernel_size=5),
     # nn.BatchNorm2d(32),
      nn.ReLU(),
      #nn.MaxPool2d(2,stride=2)
    )

    self.conv2 = nn.Sequential(
      nn.Conv2d(32, 128, kernel_size=3,padding=1,stride=1),
      #nn.ReLU(),
      #nn.Conv2d(128, 256, kernel_size=2),
      #nn.BatchNorm2d(128),
      nn.ReLU(),
     #nn.MaxPool2d(2)
    )

    self.conv3 = nn.Sequential(
      nn.Conv2d(128, 384, kernel_size=2,padding=0,stride=1),
      nn.BatchNorm2d(384),
      nn.ReLU(),
     # nn.MaxPool2d(2)
    )
    self.conv4 = nn.Sequential(
      nn.Conv2d(384, 384, kernel_size=2,padding=0,stride=1),
      nn.BatchNorm2d(384),
      nn.ReLU(),
      #nn.MaxPool2d(2)
    )
    self.fc = nn.Sequential(
      nn.Dropout(0.5),
      nn.Linear(13824,1024),
      nn.ReLU(),
      nn.Linear(1024, 1024),
      #nn.ReLU(),
      nn.Linear(1024, self.K),
      nn.ReLU()
    )

  def forward(self,X):
   # X = torch.unsqueeze(X,1)

    out = self.conv1(X)

    out = self.conv2(out)

    out = self.conv3(out)

    out = self.conv4(out)
    #out = self.conv5(out)

    #out, _  = torch.max(out, -1)
    out = out.view(out.size(0), -1)
    out = self.fc(out)
    return out



In [ ]:
class ChessModel(nn.Module):
  def __init__(self, hidden_outputs, n_outputs,device):
    super(ChessModel,self).__init__()

    self.HO=hidden_outputs
    self.K = n_outputs
    self.device=device
    #self.cnn = CNN(n_inputs=12, n_outputs=self.HO)
    self.chessDecoder=ChessDecoder()
    self.fcPlayer = nn.Sequential(
        nn.Linear(1,64),
        nn.ReLU(),
        nn.Linear(64,64),
        nn.ReLU(),
        nn.Linear(64,32),
        nn.ReLU())
    self.fc = nn.Sequential(
        nn.Linear(self.HO+32,4096),
        nn.ReLU(),
        nn.Linear(4096,2048),
        nn.ReLU(),
        nn.Linear(2048,2048),
        nn.ReLU(),
        nn.Linear(2048,self.K))

  def forward(self,X):
    #cnn_outputs = self.cnn(X[0])
    player_outputs = self.fcPlayer(X[1])

    out = torch.cat((X[0].view(X[0].size(0),-1),player_outputs),1)
    out = self.fc(out)
    #out = torch.softmax(out,dim=1)
    #mask=self.chessDecoder.getFromPositionMask(X[0],X[1],out.size(0))
    #out=out*mask
    #out = out / out.sum(dim=1, keepdim=True)
    return out

In [ ]:
model = ChessModel(
            hidden_outputs = 512,
            n_outputs=number_of_classes,
            device=device)
model.to(device)

ChessModel(
  (cnn): CNN(
    (conv1): Sequential(
      (0): Conv2d(12, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (1): ReLU()
    )
    (conv2): Sequential(
      (0): Conv2d(32, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
    )
    (conv3): Sequential(
      (0): Conv2d(128, 384, kernel_size=(2, 2), stride=(1, 1))
      (1): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (conv4): Sequential(
      (0): Conv2d(384, 384, kernel_size=(2, 2), stride=(1, 1))
      (1): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (fc): Sequential(
      (0): Dropout(p=0.5, inplace=False)
      (1): Linear(in_features=13824, out_features=1024, bias=True)
      (2): ReLU()
      (3): Linear(in_features=1024, out_features=1024, bias=True)
      (4): Linear(in_features=1024, out_features=512, bias=True)
      (5): ReLU()
    )
  )
  (fcP

In [ ]:
def CrossEntropyLoss(predictions, targets, X):
    #print(f"prd {predictions}")
    if len(targets.shape) > 1 and targets.shape[1] > 1:
        targets = torch.argmax(targets, dim=1)

    # Apply log-softmax for numerical stability
    outputs=F.softmax(predictions,dim=1)
    mask=chessDecoder.getFromPositionMask(X[0],X[1],outputs.size(0))
    #outputs=outputs*mask
    outputs = outputs * mask
    outputs = outputs.masked_fill(outputs == 0, 1e-8)  # Replace zeros with a small positive value
    row_sums = outputs.sum(dim=1, keepdim=True)
    outputs = outputs / row_sums

    #print(outputs.sum(dim=1, keepdim=True))
    #outputs = outputs / outputs.sum(dim=1, keepdim=True)
    log_probs = torch.log(outputs)
    # Gather log probabilities corresponding to the true class labels
    loss = -log_probs[range(predictions.shape[0]), targets]
    # Calculate the mean loss
    return loss.mean()

In [ ]:
criterion = nn.CrossEntropyLoss()
chessDecoder=ChessDecoder()
l=0
for i, k in train_loader:
 # print(i[0].shape)
 # i = i.view(i.size(0),1,i.size(1),i.size(2))

  l+=1
  i = i[0].reshape((-1,12,8,8)).to(device),i[1].to(device).view(-1,1)
  #print(k[0][34])
  out = model(i)
  loss = criterion(out,k.to(device)).item()
  #print(loss)
  print(CrossEntropyLoss(out,k.to(device),i).item())
  if np.isnan(criterion(out,k.to(device)).item()):
    print(out[0])
  #break
  #print(criterion(out,k.to(device)).item())
  #print(type(criterion(out,k.to(device)).item()))
  break
  #print(np.max(lak[0].cpu().detach().numpy()))
  #print(lak.size())


2.2229788303375244


# Train model

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.0001)

In [ ]:
def save_model_checkpoint(val_loss, best_val_loss, train_loss, it, model=model, optimizer=optimizer,path='model_checkpoint.pt'):
    torch.save({
            'epoch': it,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': train_loss,
            'val_loss': val_loss,
            }, path)
    print(f'Model saved at epoch: {it+1}, val_loss improved from: {best_val_loss:.4f} to: {val_loss:.4f}')

In [ ]:
def plot_loss_curves(train_losses,val_losses,train_losses2=None,val_losses2=None):
  if(train_losses2 is not None):
    train_losses = np.concatenate((train_losses,train_losses2))
    val_losses = np.concatenate((val_losses,val_losses2))
  plt.plot(train_losses,label='Train loss')
  plt.plot(val_losses,label='Val loss')
  plt.legend(); plt.show
best_val_loss=10**5

In [ ]:
epoches=50
train_losses = np.zeros(epoches)
val_losses = np.zeros(epoches)

for it in range(epoches):
  t0 = datetime.now()
  model.train() # set model to train mode
  train_loss=[]
  val_loss=[]

  # train
  for inputs,targets in train_loader:
    # move data to gpu
    inputs= inputs[0].reshape((-1,12,8,8)).to(device),inputs[1].view(-1,1).to(device)

    targets = targets.to(device)
    #inputs = inputs.permute(0,2,1)
    # zero gradients
    optimizer.zero_grad()

    # forward pass
    outputs = model(inputs)
    loss = CrossEntropyLoss(outputs,targets,inputs)
    #print(inputs)
    # backward
    loss.backward()
    optimizer.step()
    train_loss.append(loss.item())
  model.eval() # set model to eval mode
  for inputs,targets in test_loader:
    # move data to gpu
    inputs= inputs[0].reshape((-1,12,8,8)).to(device),inputs[1].view(-1,1).to(device)
    targets = targets.to(device)
   # inputs = inputs.permute(0,2,1)


    # forward pass
    outputs = model(inputs)
    loss = CrossEntropyLoss(outputs,targets,inputs)

    val_loss.append(loss.item())

  # calculate loss
  train_loss = np.mean(train_loss)
  val_loss = np.mean(val_loss)
  if (val_loss<best_val_loss):
    save_model_checkpoint(val_loss,best_val_loss,train_loss,it, path='model_checkpoint.pt')
    best_val_loss=val_loss
  # append loss
  train_losses[it]=train_loss
  val_losses[it]=val_loss
  dt = datetime.now() - t0
  print(f"Epoch {it+1}/{epoches}, Train loss: {train_loss:.4f}, Val loss: {val_loss:.4f}, Duration: {dt}")
  print('-------------------------------------------------------------')

Model saved at epoch: 1, val_loss improved from: 100000.0000 to: 1.9396
Epoch 1/50, Train loss: 1.9683, Val loss: 1.9396, Duration: 0:05:37.111427
-------------------------------------------------------------
Model saved at epoch: 2, val_loss improved from: 1.9396 to: 1.8567
Epoch 2/50, Train loss: 1.9129, Val loss: 1.8567, Duration: 0:05:40.591297
-------------------------------------------------------------
Model saved at epoch: 3, val_loss improved from: 1.8567 to: 1.8255
Epoch 3/50, Train loss: 1.8508, Val loss: 1.8255, Duration: 0:05:38.023115
-------------------------------------------------------------
Model saved at epoch: 4, val_loss improved from: 1.8255 to: 1.8065
Epoch 4/50, Train loss: 1.8103, Val loss: 1.8065, Duration: 0:05:40.482051
-------------------------------------------------------------
Model saved at epoch: 5, val_loss improved from: 1.8065 to: 1.7524
Epoch 5/50, Train loss: 1.7679, Val loss: 1.7524, Duration: 0:05:40.294372
-------------------------------------